# Creación Data referente a Juzgados de Garantía

Se realiza un analisis mediante expresiones regulares del art. 16 del Código Organico de Tribunales (COT). En este artículo se definen las comunas de Chile donde tendrán asiento este tipo de Juzgados y además su territorio de jurisdiccion.

In [6]:
import re
import os
import pandas as pd
import numpy as np

from tqdm import tqdm
from unicodedata import normalize

from pjud.data import cleandata

In [7]:
tqdm.pandas()

In [ ]:
path_raw = "./data/raw/cot"

## Articulo 16 COT ref: juzgados de garantías


In [8]:
with open(f'{path_raw}/Juzgados_Garantia.txt', 'r') as file:
    contenido_jg = ''
    for line in file.readlines():
        contenido_jg += line

Se construye una expresion regular para captar una lista con la información que se desea procesar, y así generar un dataframe.

In [9]:
regex_jg=r"(?:(?P<Region>^[\w \']+)\:\n\n)|(?P<JG>^[\w. \-]+)\,\scon\s(?P<Jueces>[\w.\-]+)[a-z\-\s\,]+(?P<Competencia>\.|\s[\w. \-\,]+)"
matches = re.findall(regex_jg, contenido_jg, re.MULTILINE)

#matches

# GENERA UNA ARREGLO DE LISTA DE 4 ELEMENTOS
# ELEMENTO 0 : REGIÓN
# ELEMENTO 1: JUZGADO DE GARANTÍA DE ...
# ELEMENTO 2: Cantidad de Jueces en Letras
# ELEMENTO 3: Competencia (. es lo mismo de 3 0 7) o varias ciudades (separadas por , o y)

#len(matches)

In [10]:
data_jg = []

for item in range(0,len(matches)):
    if matches[item][0] != '':
        region = matches[item][0].upper()
    else:    
        if matches[item][1] != '':
            ciudad = matches[item][1].upper() 
            if ciudad.find("JUZGADO") != -1: 
                juzgado = ciudad
                
            else:
                juzgado = f"JUZGADO DE GARANTIA {ciudad}"

        if matches[item][2] != '':
            cantidad_jueces = cleandata.transforma_numero(matches[item][2])
    
        if matches[item][3] == '.':
            competencia = ciudad
        
        else:  
            if matches[item][3] != '':
                competencia = matches[item][3].upper()
        
        competencia = competencia.replace(" Y ",",")
        competencia = competencia.replace(" E ",",")
        competencia = competencia.replace(".","")
        
        comunas = competencia.split(",")
        
        for comuna in comunas:
            data_jg.append([region,juzgado,ciudad,cantidad_jueces,comuna.strip(),'GARANTIA'])
   
df_juzgados_garantia = pd.DataFrame(data_jg, columns = ['REGION','TRIBUNAL','ASIENTO','JUECES','COMUNA','TIPO JUZGADO'])


In [11]:
# Elimino tildes de las columnas object

cols = df_juzgados_garantia.select_dtypes(include = ["object"]).columns
df_juzgados_garantia[cols] = df_juzgados_garantia[cols].progress_apply(cleandata.elimina_tilde)

100%|██████████| 6/6 [00:00<00:00, 243.56it/s]


In [12]:
df_juzgados_garantia

,REGION,TRIBUNAL,ASIENTO,JUECES,COMUNA,TIPO JUZGADO
0,PRIMERA REGION DE TARAPACA,JUZGADO DE GARANTIA IQUIQUE,IQUIQUE,7,IQUIQUE,GARANTIA
1,SEGUNDA REGION DE ANTOFAGASTA,JUZGADO DE GARANTIA TOCOPILLA,TOCOPILLA,1,TOCOPILLA,GARANTIA
2,SEGUNDA REGION DE ANTOFAGASTA,JUZGADO DE GARANTIA CALAMA,CALAMA,4,CALAMA,GARANTIA
3,SEGUNDA REGION DE ANTOFAGASTA,JUZGADO DE GARANTIA CALAMA,CALAMA,4,OLLAGUE,GARANTIA
4,SEGUNDA REGION DE ANTOFAGASTA,JUZGADO DE GARANTIA CALAMA,CALAMA,4,SAN PEDRO DE ATACAMA,GARANTIA
...,...,...,...,...,...,...
233,REGION DE ÑUBLE,JUZGADO DE GARANTIA CHILLAN,CHILLAN,4,CHILLAN VIEJO,GARANTIA
234,REGION DE ÑUBLE,JUZGADO DE GARANTIA YUNGAY,YUNGAY,1,EL CARMEN,GARANTIA
235,REGION DE ÑUBLE,JUZGADO DE GARANTIA YUNGAY,YUNGAY,1,PEMUCO,GARANTIA
236,REGION DE ÑUBLE,JUZGADO DE GARANTIA YUNGAY,YUNGAY,1,YUNGAY,GARANTIA


In [14]:
# Reset el index para realizar feather

df_juzgados_garantia.reset_index(inplace = True)
path_interim = "../data/interim/pjud"
os.makedirs(path_interim, exist_ok = True) 

# Guardamos dataset como archivo feather
df_juzgados_garantia.to_feather(f'{path_interim}/generates_JuzgadosGarantia.feather')
